In [1]:
import requests, praw, re, time
from bs4 import BeautifulSoup 
from datetime import datetime
import pandas as pd 
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import nltk
from nltk.corpus import stopwords 
from config import indeed_query,r_client_id,r_secret_key,r_dev,r_username,r_pw

In [2]:
reddit = praw.Reddit(
    client_id=r_client_id,
    client_secret=r_secret_key,
    password=r_pw,
    user_agent=f'testscript by u/{r_dev}',
    username=r_dev,
)
print(reddit.user.me())

Jazzy_Danger_1230


In [3]:
def getsymb(s):
    lst = []
    s = re.sub('[^A-Z\s]','', s)   
    for w in s.split(' '):  
        if re.match('\w{2,4}',w): 
            lst.append(w)
    return ','.join(lst)
getsymb('1000$ of BB will take it to 300 billion market. TO THE MOON to see SPY in the 400-405 range Bb & NOK ready for takeoff')

'BB,TO,THE,MOON,SPY,NOK'

In [14]:
submissions=reddit.subreddit("wallstreetbets").search("Daily Discussion Thread", limit=12, sort='new')

In [15]:
lst = []
for submission in submissions:
    # print(submission.title)    
    # submission.comments.replace_more(limit=None)
    submission.comment_sort = 'new'
    for comment in submission.comments: 
        try:
            lst.append(
                {
                    'submission_title':submission.title
                    ,'comment_body':comment.body
                    ,'time':datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
                    ,'author': comment.author
                    ,'score':comment.score
                    ,'permalink':comment.permalink
                    ,'replies':comment.replies
                    ,'id':comment.id 
                }
            ) 
        except:
            # print('err' + comment.id  )
            pass    
df = pd.DataFrame(lst)
df['SYMB'] = df.comment_body.apply(getsymb)  
df['time'] = pd.to_datetime(df.time)
df['day'] = df.time.apply(lambda s: s.strftime('%d'))
df['hour'] = df.time.apply(lambda s: s.strftime('%I')) 
df.permalink = 'https://www.reddit.com/' + df.permalink
df.sort_values('time', ascending=False)

,submission_title,comment_body,time,author,score,permalink,replies,id,SYMB,day,hour
1,"Daily Discussion Thread for June 08, 2021","About to go red on my chart boys, had a good run",2021-06-08 19:11:46,putsandpizza74,1,https://www.reddit.com//r/wallstreetbets/comme...,(),h12afcu,,08,07
2,"Daily Discussion Thread for June 08, 2021",If you like to fill gaps with your massive don...,2021-06-08 19:11:43,tvvt,1,https://www.reddit.com//r/wallstreetbets/comme...,(),h12af4u,TAL,08,07
3,"Daily Discussion Thread for June 08, 2021",WISH upon a tendie,2021-06-08 19:11:42,jonfnhhs,1,https://www.reddit.com//r/wallstreetbets/comme...,(),h12af2q,WISH,08,07
4,"Daily Discussion Thread for June 08, 2021",wishing $WISH to the got damn moon.,2021-06-08 19:11:41,dire_bear_,1,https://www.reddit.com//r/wallstreetbets/comme...,(),h12aezk,WISH,08,07
5,"Daily Discussion Thread for June 08, 2021",What do we think about a TSLA 615 call for Fri...,2021-06-08 19:11:40,bluewater03c,1,https://www.reddit.com//r/wallstreetbets/comme...,(),h12aex3,TSLA,08,07
...,...,...,...,...,...,...,...,...,...,...,...
2063,"Daily Discussion Thread for May 28, 2021",You could panic. Or you could…![gif](emote|fre...,2021-05-28 20:17:28,CreepyBet8062,4,https://www.reddit.com//r/wallstreetbets/comme...,(),gzss4nn,,28,08
2064,"Daily Discussion Thread for May 28, 2021",[AMC know what will happen in weeks to come ](...,2021-05-28 20:17:18,lilfigure,3,https://www.reddit.com//r/wallstreetbets/comme...,(gzus83o),gzss3w7,"AMC,DOGSTOCKHUNTER",28,08
2065,"Daily Discussion Thread for May 28, 2021",Easy hold AMC. 50+ next week. Trolls on here h...,2021-05-28 20:17:08,Easy_Jello_7024,0,https://www.reddit.com//r/wallstreetbets/comme...,(gzssd9y),gzss369,AMC,28,08
2066,"Daily Discussion Thread for May 28, 2021",Would love to see SPY in the 400-405 range nex...,2021-05-28 20:16:55,JpowYellen3some,5,https://www.reddit.com//r/wallstreetbets/comme...,(gzst1kp),gzss28b,SPY,28,08


ser = df.author.value_counts()
names = [r for r in list(ser.index.values)] 
df1 = df.loc[ df['author'].isin( names[:5] )  ]  
df1.author = df1.author.apply(lambda x: str(x.name))
df1=df1.groupby('author').describe()

In [13]:
df1=df 
df1 = df1.loc[ df1['SYMB'].str.contains('CLOV,')  ]
df1.groupby('day').count()

,submission_title,comment_body,time,author,score,permalink,replies,id,SYMB,hour
day,,,,,,,,,,
01,2,2,2,2,2,2,2,2,2,2
06,1,1,1,1,1,1,1,1,1,1
07,18,18,18,18,18,18,18,18,18,18
08,16,16,16,16,16,16,16,16,16,16
28,1,1,1,1,1,1,1,1,1,1


,submission_title,comment_body,time,author,score,permalink,replies,id,SYMB,day,hour
0,"Daily Discussion Thread for June 04, 2021",I’m a in on CLNE for a thousand Bob.,2021-06-04 13:48:36,TrioDoge,1,https://www.reddit.com//r/wallstreetbets/comme...,(),h0kf7yj,CLNE,04,01
2,"Daily Discussion Thread for June 04, 2021",What the hell did you guys do to $BB?,2021-06-04 13:48:35,NewbieDoobie18,1,https://www.reddit.com//r/wallstreetbets/comme...,(),h0kf7vi,BB,04,01
1,"Daily Discussion Thread for June 04, 2021","So, BB? Hmmmph",2021-06-04 13:48:35,Live-Ad6746,1,https://www.reddit.com//r/wallstreetbets/comme...,(),h0kf7vs,BB,04,01
3,"Daily Discussion Thread for June 04, 2021",Dont sleep on CLOV !!,2021-06-04 13:48:34,DependentHot1829,1,https://www.reddit.com//r/wallstreetbets/comme...,(),h0kf7u5,CLOV,04,01
6,"Daily Discussion Thread for June 04, 2021",Cmon bb kick girl,2021-06-04 13:48:34,OneEyedKing808,1,https://www.reddit.com//r/wallstreetbets/comme...,(),h0kf7sb,,04,01
...,...,...,...,...,...,...,...,...,...,...,...
4586,"Daily Discussion Thread for May 18, 2021",It’s 12/31/21. SPY finishes the year down 0.2%.,2021-05-18 19:55:59,Griffinhunters,5,https://www.reddit.com//r/wallstreetbets/comme...,(),gylyl21,SPY,18,07
4587,"Daily Discussion Thread for May 18, 2021",SPY calls at the buzzer??,2021-05-18 19:55:59,EvaUnit343,2,https://www.reddit.com//r/wallstreetbets/comme...,(gylyp0v),gylyl1v,SPY,18,07
4588,"Daily Discussion Thread for May 18, 2021",W H Y,2021-05-18 19:55:58,lizlegit000,3,https://www.reddit.com//r/wallstreetbets/comme...,(),gylykxb,,18,07
4589,"Daily Discussion Thread for May 18, 2021",AMC is dripping with wack juice. Buying puts t...,2021-05-18 19:55:57,Pester_Stone,-1,https://www.reddit.com//r/wallstreetbets/comme...,(),gylyku6,AMC,18,07


https://www.reddit.com//r/wallstreetbets/comments/nqi9f6/daily_discussion_thread_for_june_02_2021/h0f9a6d/
https://www.reddit.com//r/wallstreetbets/comments/nqi9f6/daily_discussion_thread_for_june_02_2021/h0d6su8/
https://www.reddit.com//r/wallstreetbets/comments/npqenw/daily_discussion_thread_for_june_01_2021/h09b3a7/
https://www.reddit.com//r/wallstreetbets/comments/npqenw/daily_discussion_thread_for_june_01_2021/h096l5t/
https://www.reddit.com//r/wallstreetbets/comments/npqenw/daily_discussion_thread_for_june_01_2021/h08ch4b/
https://www.reddit.com//r/wallstreetbets/comments/npbyt0/what_are_your_moves_tomorrow_june_01_2021/h06nrrl/
https://www.reddit.com//r/wallstreetbets/comments/npbyt0/what_are_your_moves_tomorrow_june_01_2021/h06ilnh/
https://www.reddit.com//r/wallstreetbets/comments/npbyt0/what_are_your_moves_tomorrow_june_01_2021/h06dw1z/


In [ ]:
p=0
url = f'https://washingtondc.craigslist.org/d/bicycles/search/bia?s={p}'
soup = BeautifulSoup(requests.get(url).text, "html.parser")

   
    